In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from google.colab import drive
import math
import requests


import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow_datasets as tfds 

from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/Google_Colab/NYCFC_Merch_Sentiment_Analysis'
os.chdir(os.path.join(path, 'Review_Sentiment_Data'))

In [ ]:
data = pd.read_csv('Reddit_Data.csv')
data.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [ ]:
os.chdir('..')

In [ ]:
data.describe(include='all')

,clean_comment,category
count,37149,37249.000000
unique,36799,NaN
top,,NaN
freq,115,NaN
mean,NaN,0.202771
std,NaN,0.778515
min,NaN,-1.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,1.000000


In [ ]:
data.isna().sum()

clean_comment    100
category           0
dtype: int64

In [ ]:
data[data.isna().any(axis=1)]

,clean_comment,category
413,NaN,0
605,NaN,0
2422,NaN,0
2877,NaN,0
3307,NaN,0
...,...,...
35975,NaN,0
36036,NaN,0
37043,NaN,0
37111,NaN,0


In [ ]:
data.dropna(axis=0, inplace=True)

In [ ]:
data = data.iloc[:-500, :]

In [ ]:
from tensorflow.keras.preprocessing import text

tokenizer = text.Tokenizer(num_words=1000)
tokenizer.fit_on_texts(data['clean_comment'])
print(tokenizer.word_index)

{'the': 1, 'and': 2, 'that': 3, 'this': 4, 'for': 5, 'you': 6, 'are': 7, 'not': 8, 'they': 9, 'have': 10, 'with': 11, 'but': 12, 'will': 13, 'was': 14, 'people': 15, 'what': 16, 'india': 17, 'all': 18, 'has': 19, 'like': 20, 'bjp': 21, 'can': 22, 'from': 23, 'modi': 24, 'about': 25, 'just': 26, 'there': 27, 'who': 28, 'their': 29, 'one': 30, 'his': 31, 'how': 32, 'more': 33, 'good': 34, 'would': 35, 'your': 36, 'now': 37, 'don': 38, 'them': 39, 'out': 40, 'when': 41, 'some': 42, 'even': 43, 'same': 44, 'get': 45, 'any': 46, 'because': 47, 'been': 48, 'why': 49, 'think': 50, 'only': 51, 'other': 52, 'time': 53, 'which': 54, 'these': 55, 'congress': 56, 'should': 57, 'our': 58, 'also': 59, 'were': 60, 'than': 61, 'government': 62, 'him': 63, 'indian': 64, 'know': 65, 'see': 66, 'here': 67, 'had': 68, 'country': 69, 'right': 70, 'then': 71, 'much': 72, 'its': 73, 'being': 74, 'well': 75, 'party': 76, 'very': 77, 'way': 78, 'did': 79, 'most': 80, 'make': 81, 'lot': 82, 'really': 83, 'after

In [ ]:
tokenized_data = tokenizer.texts_to_matrix(data['clean_comment'])

In [ ]:
tokenized_data.shape

(36649, 1000)

In [ ]:
data[data['category'] == -1] = 2

In [ ]:
data['category'].value_counts()

1    15621
0    12831
2     8197
Name: category, dtype: int64

In [ ]:
data['category'].shape

(36649,)

In [ ]:
MAX_LEN = max([len(sentence) for sentence in tokenized_data])

input_data = tf.keras.preprocessing.sequence.pad_sequences(tokenized_data, 
                                                           maxlen=MAX_LEN, padding='post')

In [ ]:
print(input_data.shape)
print(data['category'].shape)

(36649, 1000)
(36649,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(input_data, data['category'], test_size=0.3,
                                                    random_state=101)

In [ ]:
model = Sequential()

model.add(Embedding(1000, 32, input_length=MAX_LEN))
model.add(Dropout(0.2))

model.add(Conv1D(64, 2, padding='same', activation='relu'))
model.add(MaxPool1D())

model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPool1D())

model.add(Conv1D(64, 4, padding='same', activation='relu'))
model.add(MaxPool1D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))

model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1000, 32)          32000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 32)          0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 1000, 64)          4160      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 500, 64)           0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 500, 64)           12352     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 250, 64)           0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 250, 64)          

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
tensorboard = TensorBoard(log_dir='logs/')

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='sparse_categorical_crossentropy',
                  metrics=['sparse_categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test),
              callbacks=[early_stopping, tensorboard])

Epoch 1/10
  2/401 [..............................] - ETA: 26s - loss: 1.0672 - sparse_categorical_accuracy: 0.4531WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0207s vs `on_train_batch_end` time: 0.1142s). Check your callbacks.


401/401 [==============================] - 6s 16ms/step - loss: 0.8056 - sparse_categorical_accuracy: 0.6397 - val_loss: 0.6449 - val_sparse_categorical_accuracy: 0.7410
Epoch 2/10
401/401 [==============================] - 6s 15ms/step - loss: 0.5780 - sparse_categorical_accuracy: 0.7776 - val_loss: 0.5821 - val_sparse_categorical_accuracy: 0.7778
Epoch 3/10
401/401 [==============================] - 6s 15ms/step - loss: 0.4751 - sparse_categorical_accuracy: 0.8247 - val_loss: 0.5959 - val_sparse_categorical_accuracy: 0.7762
Epoch 4/10
401/401 [==============================] - 6s 15ms/step - loss: 0.3609 - sparse_categorical_accuracy: 0.8712 - val_loss: 0.6420 - val_sparse_categorical_accuracy: 0.7825
Epoch 5/10
401/401 [==============================] - 6s 15ms/step - loss: 0.2359 - sparse_categorical_accuracy: 0.9204 - val_loss: 0.8257 - val_sparse_categorical_accuracy: 0.7601
Epoch 6/10
401/401 [==============================] - 6s 15ms/step - loss: 0.1546 - sparse_categorical_acc

In [ ]:
model.save('CNN_Text_Model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: CNN_Text_Model/assets


INFO:tensorflow:Assets written to: CNN_Text_Model/assets
